In [90]:
from internetarchive import download
from internetarchive import search_items
import pandas as pd
import py7zr
from os import listdir
import logging
import numpy as np
import pyodbc
import sqlalchemy

In [91]:
def download_stackexchange(name):
    # set filename and download .7z from archive
    filename = name + '.stackexchange.com.7z'
    download('stackexchange', files=filename, verbose=True)
    
    # unpack archive
    logging.info(f'Unpacking {filename}')
    with py7zr.SevenZipFile('stackexchange/' + filename, 'r') as archive:
        archive.extractall(path=f"./stackexchange/{name}")
    
    # discover all XML files present in download and parse
    tables = [name.split(sep='.')[0] for name in listdir(f'stackexchange/{name}')]
    df = {table.lower():pd.read_xml(f'stackexchange/{name}/{table}.xml', parser='etree').rename(columns=lambda col: f'{table.lower()}_{col.lower()}') for table in tables}
    df['posts']['posts_stackexchangename'] = name
    return df

In [176]:
name = 'woodworking'
woodworking = download_stackexchange(name)

stackexchange:
 skipping stackexchange\woodworking.stackexchange.com.7z, file already exists based on length and date.


In [109]:
name = 'ai'
ai = download_stackexchange(name)

stackexchange:
 skipping stackexchange\ai.stackexchange.com.7z, file already exists based on length and date.


In [56]:
for key, value in woodworking.items():
    print(key + str(value.columns.to_list()))

Badges['Class', 'Date', 'Id', 'Name', 'TagBased', 'UserId']
Comments['ContentLicense', 'CreationDate', 'Id', 'PostId', 'Score', 'Text', 'UserId', 'UserDisplayName']
PostHistory['ContentLicense', 'CreationDate', 'Id', 'PostHistoryTypeId', 'PostId', 'RevisionGUID', 'Text', 'UserId', 'UserDisplayName', 'Comment']
PostLinks['CreationDate', 'Id', 'LinkTypeId', 'PostId', 'RelatedPostId']
Posts['AnswerCount', 'Body', 'CommentCount', 'ContentLicense', 'CreationDate', 'FavoriteCount', 'Id', 'LastActivityDate', 'LastEditDate', 'LastEditorUserId', 'OwnerUserId', 'PostTypeId', 'Score', 'Tags', 'Title', 'ViewCount', 'AcceptedAnswerId', 'OwnerDisplayName', 'ParentId', 'ClosedDate', 'LastEditorDisplayName', 'CommunityOwnedDate', 'StackExchangeName']
Tags['Count', 'ExcerptPostId', 'Id', 'TagName', 'WikiPostId']
Users['AboutMe', 'AccountId', 'CreationDate', 'DisplayName', 'DownVotes', 'Id', 'LastAccessDate', 'Location', 'Reputation', 'UpVotes', 'Views', 'ProfileImageUrl', 'WebsiteUrl']
Votes['CreationD

In [83]:
measurer = np.vectorize(len)
print(measurer(woodworking[table].values.astype(str)).max(axis=0))
print(measurer(ai[table].values.astype(str)).max(axis=0))

[4566    8   23   36    3    5   23  100    5    4    3  128  157]
[6182   10   23   36    4    5   23  100    5    4    4  128  200]


In [95]:
user = 'admin'
password = 'jf6qDce3CKCrDRrM'
server = 'ticketswap-stackexchange-dwh.csboebiw4ypn.eu-west-3.rds.amazonaws.com'
port = '1433'
database = 'ticketswap-stackexchange-dwh'
driver = 'SQL Server Native Client 11.0'

In [105]:
cnxn_str = ("Driver={SQL Server Native Client 11.0};"
            "Server=ticketswap-stackexchange-dwh.csboebiw4ypn.eu-west-3.rds.amazonaws.com,1433;"
            "Database=ticketswap-stackexchange-dwh;"
            "UID=admin;"
            "PWD=jf6qDce3CKCrDRrM;")
cnxn = pyodbc.connect(cnxn_str)

In [128]:
tables_with_ids = {'badges':'badgeid', 
                    'comments':'commentid', 
                    'posthistory':'posthistoryid', 
                    'postlinks': 'postlinkid',
                    'posts': 'postid',
                    'tags': 'tagid',
                    'users': 'userid',
                    'votes': 'voteid'}

In [169]:
def reset_indexes(archive, tables_with_ids, con):
    columns_to_add = {}
    for tablename, table in archive.items():
        id_name = tables_with_ids[tablename]
        max_index = pd.read_sql(f'SELECT MAX({tablename}_id) FROM dbo.{tablename}', con=con).iloc[0,0]
        print(max_index)
        if max_index is None:
            max_index = 1
        print(table[f'{tablename}_id'].min())
        to_add = max_index - table[f'{tablename}_id'].min()
        columns_to_add[id_name] = to_add
    for colname, value in columns_to_add.items():
        for table in archive.keys():
            for col in archive[table].columns:
                if col == f'{table}_{colname}':
                    print(f'adding {value} to {table}.{col}')
                    archive[table][col] += value   

In [170]:
reset_indexes(ai, tables_with_ids, con)

75490
75491
None
9
38416
1
141888
103
13863
1
430
2
12144
-1
54868
1
adding 13862 to comments.comments_postid
adding 13862 to posthistory.posthistory_postid
adding 13862 to postlinks.postlinks_postid
adding 13862 to votes.votes_postid
adding 12145 to badges.badges_userid
adding 12145 to comments.comments_userid
adding 12145 to posthistory.posthistory_userid
adding 12145 to votes.votes_userid


In [179]:
engine = sqlalchemy.create_engine(f'mssql+pyodbc://{user}:{password}@{server}:{port}/{database}?driver=SQL+Server+Native+Client+11.0')
con = engine.connect()

In [171]:
con.close()

In [144]:
pd.read_sql(f'SELECT MAX(comments_id) FROM dbo.comments', con=con)

,
0,None


In [149]:
result = pd.read_sql('SELECT MAX(badges_id) FROM dbo.badges', con=con).iloc[0,0]
result

75490

In [139]:
to_add = result - ai['badges']['badges_id'].min()
to_add

54457

In [140]:
ai['badges']['badges_id'] += to_add

In [175]:
woodworking['comments']
[comments_contentlicence]
[comments_contentlicense]

,comments_contentlicense,comments_creationdate,comments_id,comments_postid,comments_score,comments_text,comments_userid,comments_userdisplayname
0,CC BY-SA 3.0,2015-03-17T15:23:35.850,2,1,3,This is basically the technique I use. http:/...,33.0,None
1,CC BY-SA 3.0,2015-03-17T15:25:14.843,3,2,0,you wouldn't happen to have access to a band s...,32.0,None
2,CC BY-SA 3.0,2015-03-17T15:25:18.220,4,5,0,I'm not 100% sure this is technically in scope...,14.0,None
3,CC BY-SA 3.0,2015-03-17T15:25:59.820,5,1,3,Welcome to your new SE community! Can you give...,23.0,None
4,CC BY-SA 3.0,2015-03-17T15:26:41.430,6,7,1,I highly suggest going to a 'panel' style blad...,31.0,None
...,...,...,...,...,...,...,...,...
24113,CC BY-SA 4.0,2022-06-04T13:49:07.107,30528,13863,1,I do not believe you should use a filler betwe...,1821.0,None
24114,CC BY-SA 4.0,2022-06-04T21:40:36.267,30529,13863,0,"Thanks @AloysiusDefenestrate, your straightfor...",3133.0,None
24115,CC BY-SA 4.0,2022-06-04T21:41:29.943,30530,13863,1,Agreed that this may be off-topic for woodwork...,3133.0,None
24116,CC BY-SA 4.0,2022-06-05T01:37:21.037,30531,13863,0,I've flagged it for a mod to move it to [diy.s...,93.0,None


In [ ]:
woodworking['badges'].to_sql('badges', con=con, if_exists='append', index=False, chunksize=100, method='multi')

In [177]:
woodworking['comments'].drop(columns=['comments_text','comments_userdisplayname'], inplace=True)

In [181]:
woodworking['comments']

,comments_contentlicense,comments_creationdate,comments_id,comments_postid,comments_score,comments_userid
0,CC BY-SA 3.0,2015-03-17T15:23:35.850,2,1,3,33.0
1,CC BY-SA 3.0,2015-03-17T15:25:14.843,3,2,0,32.0
2,CC BY-SA 3.0,2015-03-17T15:25:18.220,4,5,0,14.0
3,CC BY-SA 3.0,2015-03-17T15:25:59.820,5,1,3,23.0
4,CC BY-SA 3.0,2015-03-17T15:26:41.430,6,7,1,31.0
...,...,...,...,...,...,...
24113,CC BY-SA 4.0,2022-06-04T13:49:07.107,30528,13863,1,1821.0
24114,CC BY-SA 4.0,2022-06-04T21:40:36.267,30529,13863,0,3133.0
24115,CC BY-SA 4.0,2022-06-04T21:41:29.943,30530,13863,1,3133.0
24116,CC BY-SA 4.0,2022-06-05T01:37:21.037,30531,13863,0,93.0


In [180]:
woodworking['comments'].to_sql('comments', con=con, if_exists='append', index=False, chunksize=100, method='multi')

ProgrammingError: (pyodbc.ProgrammingError) ('42S22', "[42S22] [Microsoft][SQL Server Native Client 11.0][SQL Server]Invalid column name 'comments_contentlicense'. (207) (SQLExecDirectW); [42S22] [Microsoft][SQL Server Native Client 11.0][SQL Server]Invalid column name 'comments_contentlicense'. (207); [42S22] [Microsoft][SQL Server Native Client 11.0][SQL Server]Statement(s) could not be prepared. (8180)")
[SQL: INSERT INTO comments (comments_contentlicense, comments_creationdate, comments_id, comments_postid, comments_score, comments_userid) VALUES (?, ?, ?, ?, ?, ?), (?, ?, ?, ?, ?, ?), (?, ?, ?, ?, ?, ?), (?, ?, ?, ?, ?, ?), (?, ?, ?, ?, ?, ?), (?, ?, ?, ?, ?, ?), (?, ?, ?, ?, ?, ?), (?, ?, ?, ?, ?, ?), (?, ?, ?, ?, ?, ?), (?, ?, ?, ?, ?, ?), (?, ?, ?, ?, ?, ?), (?, ?, ?, ?, ?, ?), (?, ?, ?, ?, ?, ?), (?, ?, ?, ?, ?, ?), (?, ?, ?, ?, ?, ?), (?, ?, ?, ?, ?, ?), (?, ?, ?, ?, ?, ?), (?, ?, ?, ?, ?, ?), (?, ?, ?, ?, ?, ?), (?, ?, ?, ?, ?, ?), (?, ?, ?, ?, ?, ?), (?, ?, ?, ?, ?, ?), (?,

In [172]:
woodworking['posthistory'].drop(columns=['posthistory_revisionguid', 'posthistory_text','posthistory_userdisplayname', 'posthistory_comment'], inplace=True)

In [ ]:
woodworking['posthistory'].to_sql('posthistory', con=con, if_exists='append', index=False, chunksize=100, method='multi')

In [ ]:
woodworking['postlinks'].to_sql('postlinks', con=con, if_exists='append', index=False, chunksize=200, method='multi')

In [ ]:
woodworking['posts'].drop(columns=['posts_body', 'posts_title', 'posts_tags'], inplace=True)

In [ ]:
woodworking['posts'].to_sql('posts', con=con, if_exists='append', index=False, chunksize=100, method='multi')

In [ ]:
woodworking['tags'].to_sql('tags', con=con, if_exists='append', index=False, chunksize=100, method='multi')

In [ ]:
woodworking['users'].drop(columns=['users_aboutme', 'users_location', 'users_profileimageurl', 'users_websiteurl'], inplace=True)

In [ ]:
woodworking['users'].to_sql('users', con=con, if_exists='append', index=False, chunksize=100, method='multi')

In [ ]:
woodworking['votes'].to_sql('votes', con=con, if_exists='append', index=False, chunksize=100, method='multi')